<a href="https://colab.research.google.com/github/MichelleInnovates/wk-6-ai/blob/main/Edge_AI_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -----------------------------------------------------------------
# Step 1: Setup and Data Download
# -----------------------------------------------------------------
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile

# Set up Kaggle API token
# NOTE: You must upload your 'kaggle.json' API key to the Colab session first.
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download and unzip the dataset
!kaggle datasets download -d mostafaabla/garbage-classification
!unzip -q garbage-classification.zip

# Define key parameters
img_height = 180
img_width = 180
batch_size = 32
data_dir = "garbage_classification" # Main folder unzipped

# -----------------------------------------------------------------
# Step 2: Load and Preprocess Data
# -----------------------------------------------------------------

# Create the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Create the validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Get class names
class_names = train_ds.class_names
print(f"Classes found: {class_names}")
num_classes = len(class_names)

# Configure dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# -----------------------------------------------------------------
# Step 3: Build and Train the AI Model
# -----------------------------------------------------------------

# Create a lightweight model (simple CNN)
model = keras.Sequential([
    # Rescale inputs to be between 0 and 1
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),

    # Data augmentation to prevent overfitting
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),

    # Convolutional blocks
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    # Flatten and Dense layers
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes) # Output layer (no softmax needed, see loss)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.summary()

# Train the model (using 10 epochs for a quick demo)
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

# -----------------------------------------------------------------
# Step 4: Convert the Model to TensorFlow Lite
# -----------------------------------------------------------------
print("\n--- Converting to TensorFlow Lite ---")

# Create a TFLiteConverter object from the Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply optimizations (default optimization quantizes weights)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('recyclable_model.tflite', 'wb') as f:
    f.write(tflite_model)

print(f"Model saved as 'recyclable_model.tflite'")

# -----------------------------------------------------------------
# Step 5: Compare File Sizes and Test TFLite Model
# -----------------------------------------------------------------
# Save the full Keras model for comparison
model.save('full_model.h5')

# Check file sizes
full_model_size = os.path.getsize('full_model.h5') / (1024 * 1024)
tflite_model_size = os.path.getsize('recyclable_model.tflite') / (1024 * 1024)

print(f"\n--- Model Size Comparison ---")
print(f"Full Keras Model Size:  {full_model_size:.2f} MB")
print(f"TFLite Model Size:      {tflite_model_size:.2f} MB")

# Test the TFLite model with the TFLite Interpreter
print("\n--- Testing TFLite Model ---")
interpreter = tf.lite.Interpreter(model_path='recyclable_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get one batch of test data
for images, labels in val_ds.take(1):
    image_batch = images
    label_batch = labels

# Test the first image in the batch
test_image = np.expand_dims(image_batch[0], axis=0)
interpreter.set_tensor(input_details[0]['index'], test_image)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

# Get prediction
prediction = np.argmax(tflite_results[0])
true_label = label_batch[0].numpy()

print(f"True Label:     {class_names[true_label]}")
print(f"TFLite Predict: {class_names[prediction]}")
print(f"Prediction successful: {class_names[true_label] == class_names[prediction]}")

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open garbage-classification.zip, garbage-classification.zip.zip or garbage-classification.zip.ZIP.


NotFoundError: Could not find directory garbage_classification